In [1]:
from data.fwf_dataset import FwfDataset
from models.fgf import FGFeatNetwork
from utils.metrics import get_multilevel_metrics, print_metrics, combine_metrics_list, EarlyStopping
from utils.general import generate_timestamp

from argparse import ArgumentParser
from tqdm import tqdm
from time import time
import numpy as np
import json
import os

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from omegaconf import OmegaConf

# metrics computation alternatives
from sklearn.metrics import jaccard_score
from torchmetrics.segmentation import MeanIoU






In [2]:
# build config
cfg = OmegaConf.load("./config/default.yaml")
cfg = OmegaConf.merge(cfg, OmegaConf.load(cfg.data.split))
with open(os.path.join(cfg.data.dataset_root, 'class_dict.json'),'r') as f:
    cfg = OmegaConf.merge(cfg, OmegaConf.create({'data':{'label_schema':json.load(f)}}))


print(OmegaConf.to_yaml(cfg))

exp = '2024-12-15_16-29-09(first)'
model_version = 'latest_model.pth'

output_dir = os.path.join('./exp',exp,'test')

general:
  exp_description: ''
  device: cuda
  batch_size: 1024
  weight_decay: 0.0005
  max_epochs: 50
  early_stopping_minDelta: 0.01
  num_workers: 4
data:
  dataset_root: ../../02_Datasets/FWF_Subsampled/0.01_simple/
  split: ./config/default_split.yaml
  preprocessing:
    _transformsTraining_:
      xyz:
        TransCenterXyz: {}
        TransZRotation: {}
        TransScaling: {}
        TransGaussianNoise:
          mean: 0
          std: 0.002
      rgb:
        TransGaussianNoise:
          mean: 0
          std: 0.02
        TransFeatureDropout:
          dropout_prob: 0.1
      wfm:
        TransSignalScaling: {}
      normals:
        TransGaussianNoise:
          mean: 0
          std: 0.05
      incAngles:
        TransGaussianNoise:
          mean: 0
          std: 0.05
        TransStandardize:
          mean: 2.07
          std: 0.38
      distanceFromScanner:
        TransGaussianNoise:
          mean: 12.13
          std: 12.2
    _transformsValidation_:
      xyz

In [3]:
return_full_probs = True

In [5]:
import pandas as pd
from plyfile import PlyData, PlyElement
from torch.nn.functional import softmax

for test_project in cfg.data._testProjects_:
    proj_name = test_project['proj_name']
    print(proj_name)
    test_ds = FwfDataset(cfg, cfg.data.preprocessing._transformsValidation_, [test_project],
                         return_resiIdx=True, return_projIdx=True)
    test_ds.subsample_grid(0.02, save_inv=True)
    test_ds.compute_neibors_knn(k=cfg.data.num_neib_normalsComputation)
    test_ds.compute_normals_knn()
    test_ds.compute_incAngles()
    test_ds.compute_neibors_knn(k=cfg.data.num_neib_featureExtraction, verbose=True)
    model = FGFeatNetwork(cfg=cfg,
    num_input_feats = test_ds[0]['features_neibors'].shape[-1],
    ).to(device=cfg.general.device)
    model.load_state_dict(torch.load(os.path.join('./exp',exp,model_version), weights_only=True))
    
    test_dl = DataLoader(test_ds, batch_size=cfg.testing.batch_size, num_workers=cfg.general.num_workers)

    

    # initiate containers
    all_preds_sub = [dict() for p in test_ds.projects]

    if return_full_probs:
        all_probs_sub = [dict() for p in test_ds.projects]


    for pi, proj in enumerate(test_ds.projects):
        for label_name in test_ds.label_names:

            all_preds_sub[pi][label_name] = torch.ones(size = (test_ds.projects[pi]['xyz_sub'].shape[0],)).to(
                device=cfg.general.device,dtype=torch.int64)*-1
            if return_full_probs:
                all_probs_sub[pi][label_name] = torch.ones(size = (test_ds.projects[pi]['xyz_sub'].shape[0],
                    len(cfg.data.label_schema[label_name]))).to(device=cfg.general.device,dtype=torch.float32)*-1
                
    
    # run test loop
    with torch.no_grad():
        for batch in tqdm(test_dl):
            # put batch on device
            for k, v in batch.items():
                batch[k] = v.to(device=cfg.general.device)
            out = model(batch)
            probs = dict()
            for k,v in out.items():
                probs[k] = softmax(v,dim=-1)
                preds = torch.argmax(probs[k], dim=-1)
                all_preds_sub[0][k][batch['residual_idx']] = preds # assume dataset has one project inside !

                all_probs_sub[0][k][batch['residual_idx']] = probs[k]
        # convert to numpy and upsample
        for k in cfg.data.label_names:
            all_preds_sub[0][k] = all_preds_sub[0][k].cpu().detach().numpy()
            all_preds_sub[0][k] = all_preds_sub[0][k][test_ds.projects[0]['sub_inv']]

            all_probs_sub[0][k] = all_probs_sub[0][k].cpu().detach().numpy()
            all_probs_sub[0][k] = all_probs_sub[0][k][test_ds.projects[0]['sub_inv']]

    

    probs = pd.DataFrame(all_probs_sub[0]['labels_3'], columns=cfg.data.label_schema['labels_3'].keys())

    xyz = pd.DataFrame(np.concatenate([test_ds.projects[0]['xyz'], all_preds_sub[0]['labels_3'][:,None]], axis=1), columns = ['x','y','z','preds_3'])
    pcd = pd.concat([xyz,probs], axis=1)
    
    PlyData([PlyElement.describe(pcd.to_records(index=False),'vertex')]).write(f"./_temp/{proj_name.replace('.FwfProj','_0.02')}.ply")

         
        

    


2023-08-28_FW_EingangBauing.FwfProj
Loading '2023-08-28_FW_EingangBauing.FwfProj'; Bounding box IDs = default
Computing neibors for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox' @ k=20
Computing normals for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox' @ k=20
Computing incidence angles for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox'
Computing neibors for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox' @ k=128


  2%|▏         | 113/5630 [00:11<09:39,  9.53it/s]


KeyboardInterrupt: 

In [7]:
# from torch.nn.functional import softmax
# # run test loop
# with torch.no_grad():
#     for batch in tqdm(test_dl):
#         # put batch on device
#         for k, v in batch.items():
#             batch[k] = v.to(device=cfg.general.device)
#         out = model(batch)
#         probs = dict()
#         for k,v in out.items():
#             probs[k] = softmax(v,dim=-1)
#             preds = torch.argmax(probs[k], dim=-1)
#             all_preds_sub[0][k][batch['residual_idx']] = preds # assume dataset has one project inside !

#             all_probs_sub[0][k][batch['residual_idx']] = probs[k]
#     # convert to numpy and upsample
#     for k in cfg.data.label_names:
#         all_preds_sub[0][k] = all_preds_sub[0][k].cpu().detach().numpy()
#         all_preds_sub[0][k] = all_preds_sub[0][k][test_ds.projects[0]['sub_inv']]

#         all_probs_sub[0][k] = all_probs_sub[0][k].cpu().detach().numpy()
#         all_probs_sub[0][k] = all_probs_sub[0][k][test_ds.projects[0]['sub_inv']]



            
        


100%|██████████| 2745/2745 [04:34<00:00, 10.01it/s]


In [70]:
all_preds_sub[0]['labels_3'][None,:].shape

(1, 10117061)

In [62]:
cfg.data.label_schema['labels_3']
all_probs_sub

[{'labels_3': array([11, 11, 11, ..., 11, 11, 11])}]

In [8]:
# import pandas as pd
# from plyfile import PlyData, PlyElement

# probs = pd.DataFrame(all_probs_sub[0]['labels_3'], columns=cfg.data.label_schema['labels_3'].keys())

# xyz = pd.DataFrame(np.concatenate([test_ds.projects[0]['xyz'], all_preds_sub[0]['labels_3'][:,None]], axis=1), columns = ['x','y','z','preds_3'])
# pcd = pd.concat([xyz,probs], axis=1)
# PlyData([PlyElement.describe(pcd.to_records(index=False),'vertex')]).write(f"./_temp/test_preds_with_probs.ply")


In [55]:
np.unique(aps, return_counts=True)

NameError: name 'aps' is not defined

In [24]:
probs[k]

tensor([[4.8074e-03, 2.2325e-05, 1.6447e-03,  ..., 3.2626e-04, 4.2728e-02,
         9.1585e-01],
        [4.1225e-03, 2.3528e-04, 1.1088e-02,  ..., 8.2548e-04, 6.7049e-02,
         7.4982e-01],
        [3.0914e-05, 5.9706e-09, 4.5178e-06,  ..., 1.6835e-07, 7.4231e-03,
         9.9031e-01],
        ...,
        [4.1427e-05, 7.4030e-09, 1.5635e-06,  ..., 1.2962e-07, 1.2515e-02,
         9.8635e-01],
        [4.0414e-04, 7.6634e-08, 2.0110e-05,  ..., 4.0212e-06, 2.7116e-02,
         9.6443e-01],
        [1.0465e-03, 2.2469e-06, 2.0309e-04,  ..., 3.0270e-05, 2.5817e-02,
         9.5214e-01]], device='cuda:0')

(10117061,)

In [146]:
(test_ds.projects[0]['labels_sub'].reshape(-1) == aps).astype(np.float32).mean()
aps[test_ds][0]['inv_inds']

KeyboardInterrupt: 

In [40]:
len(cfg.data.label_schema[label_name])

12

In [37]:
label_name

'labels_3'

In [10]:
test_ds.projects[0]['sub_inv'].shape

(10117061,)